[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/caer200/ocelotml_coupling/blob/main/eval.ipynb)

In [1]:
from ocelotml import predict_from_file, load_models

In [2]:
model =  load_models('hh')

In [3]:
model

SphereNet(
  (init_e): init(
    (emb): Embedding(95, 128)
    (lin_rbf_0): Linear(in_features=6, out_features=128, bias=True)
    (lin): Linear(in_features=384, out_features=128, bias=True)
    (lin_rbf_1): Linear(in_features=6, out_features=128, bias=False)
  )
  (init_v): update_v(
    (lin_up): Linear(in_features=128, out_features=256, bias=True)
    (lins): ModuleList(
      (0-2): 3 x Linear(in_features=256, out_features=256, bias=True)
    )
    (lin): Linear(in_features=256, out_features=1, bias=False)
  )
  (init_u): update_u()
  (emb): emb(
    (dist_emb): dist_emb(
      (envelope): Envelope()
    )
    (angle_emb): angle_emb()
    (torsion_emb): torsion_emb()
  )
  (update_vs): ModuleList(
    (0-3): 4 x update_v(
      (lin_up): Linear(in_features=128, out_features=256, bias=True)
      (lins): ModuleList(
        (0-2): 3 x Linear(in_features=256, out_features=256, bias=True)
      )
      (lin): Linear(in_features=256, out_features=1, bias=False)
    )
  )
  (update_es):

In [8]:
predict_from_file("./test.xyz",model)

KeyboardInterrupt: 